<div align="left">
<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png" align="right">
<p align="left"><b>Authors: Mikel Baraza Vidal, Daniele Borghesi, Eva Cantín Larumbe, Francesco Pio Capoccello, Eva Teisberg</b></p>
</div>
<img align="left" width="30%" src="https://www.inf.upv.es/www/etsinf/wp-content/uploads/2017/02/ETSInf_PRINCIPAL_V-horizontal.png"/> <img width="30%" src="https://www.upv.es/perfiles/pas-pdi/imagenes/marca_UPV_principal_negro150.png" align="right"/>


In [2]:
import folium
import requests
import json
import geopandas as geo
from io import StringIO
import io
import pandas as pd
#------------------------------------

from ortools.linear_solver import pywraplp
from numpy import average, std
from statistics import *

###  Datasets

In [26]:
distance = pd.read_csv("distance_stations_neighborhoods.csv", index_col=0)
neighborhood = pd.read_csv("neighborhoods_info_standardized.csv")
serviceability = pd.read_csv("neighborhoods_serviceability.csv", index_col=0)

In [27]:
distance.head()

,42 South Street,49 Beekman Street,100 Duane Street,14 N. Moore Street,75 Canal Street,25 Pitt Street,222 East 2 Street,340 East 14 Street,253 Lafayette Street,42 Great Jones Street,...,256 Hylan Blvd.,278 McClean Avenue,1850 Clove Road,1592 Richmond Road,256 Nelson Ave.,345 Annadale Road,1560 Drumgoole Road West,7219 Amboy Road,1100 Rossville Ave,305 Front Street
QN08,2123.18,2130.20,2104.71,2073.11,1896.78,1881.48,1925.06,1876.57,2030.47,1974.10,...,2449.03,2393.86,2396.74,2617.64,3444.35,3471.11,3761.95,3933.27,3559.47,2631.33
BX28,1894.39,1874.63,1935.64,2020.57,1809.95,1751.58,1728.54,1601.29,1843.17,1762.19,...,3168.88,3133.14,3051.15,3274.89,3596.90,3272.39,3250.70,3276.83,3037.71,3351.18
QN55,1872.67,1776.72,1811.35,1821.87,1646.27,1630.97,1674.55,1626.81,1779.96,1724.34,...,2113.36,2058.18,2061.06,2281.96,3108.67,3135.43,3426.27,3597.59,3223.79,2295.65
BK50,1532.61,1587.44,1614.04,1630.87,1548.42,1533.11,1576.70,1662.97,1682.11,1704.08,...,1726.71,1671.54,1674.41,1895.32,2722.02,2748.79,3039.63,3210.95,2837.15,1909.01
BX41,1774.19,1754.43,1815.43,1900.37,1689.74,1631.38,1608.33,1481.08,1722.97,1641.99,...,3048.68,3012.94,2893.91,3117.65,3439.67,3115.16,3093.46,3119.59,2880.48,3201.09


In [28]:
neighborhood.head()

,nta,boro_name,name,shape_area,population,population_density,engine_needed_first_shift,engine_needed_second_shift,ladder_needed_first_shift,ladder_needed_second_shift,...,engine_needed_per_density_first_shift,engine_needed_per_density_second_shift,ladder_needed_per_density_first_shift,ladder_needed_per_density_second_shift,rescue_needed_per_density_first_shift,rescue_needed_per_density_second_shift,squad_needed_per_density_first_shift,squad_needed_per_density_second_shift,hazardous_needed_per_density_first_shift,hazardous_needed_per_density_second_shift
0,QN08,Queens,St. Albans,0.223361,0.367078,0.109918,0.292654,0.339234,0.278632,0.342927,...,0.108119,0.088128,0.106058,0.090719,0.100143,0.082782,0.090793,0.088288,0.097795,0.105349
1,BX28,Bronx,Van Cortlandt Village,0.062467,0.378462,0.341809,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,QN55,Queens,South Ozone Park,0.239059,0.573192,0.161128,0.487559,0.479621,0.464957,0.469801,...,0.169097,0.116969,0.166145,0.116673,0.173235,0.209827,0.193908,0.076507,0.183615,0.108229
3,BK50,Brooklyn,Canarsie,0.237903,0.632227,0.178528,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,BX41,Bronx,Mount Hope,0.028422,0.391357,0.616431,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [29]:
serviceability.head()

,42 South Street,49 Beekman Street,100 Duane Street,14 N. Moore Street,75 Canal Street,25 Pitt Street,222 East 2 Street,340 East 14 Street,253 Lafayette Street,42 Great Jones Street,...,256 Hylan Blvd.,278 McClean Avenue,1850 Clove Road,1592 Richmond Road,256 Nelson Ave.,345 Annadale Road,1560 Drumgoole Road West,7219 Amboy Road,1100 Rossville Ave,305 Front Street
QN08,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
BX28,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
QN55,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
BK50,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
BX41,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Creating an instance of the Solver class

In [30]:
solver = pywraplp.Solver.CreateSolver('GLOP')

### Creating decision variables

with:

$X_{i, j, k}: $  number of vehicles of type ${i}$ deployed in station ${j}$ during shift ${k}$

$X_{i, j, k} \geq 0$, $X_{i, j, k} \in \mathbb{R}$

$ i \in I, j \in J, k \in K$

_________________

$Y_{i, a, b}:$  number of vehicles of type ${i}$ moved from station ${a}$ to station ${b}$ at shift change

$Y_{i, a, b} \geq 0$, $Y_{i, a, b} \in \mathbb{R}$

$ i \in I, a \in J, b \in J$

_____

$T_{i, j, k}: 1$ if a vehicle of type ${i}$ is deployed in station ${j}$ during shift ${k}$, $0$ otherwise

$T_{i, j, k} \geq 0$, $T_{i, j, k} \in \mathbb{Z}$

$ i \in I, j \in J, k \in K$

____


$Q_{j, k}: 1$ if there are more than two vehicles in station $j$ during shift $k$, $0$ otherwise

$Q_{j, k} \geq 0$, $Q_{j, k} \in \mathbb{Z}$

$ j \in J, k \in K$

____

- type of vehicles$: i=1\dots 5$

- stations: $j=1\dots 218$

- shifts: $k=1\dots 2$

- neighborhoods: $n=1 \dots 195$


In [92]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidents = json.loads(response.text)

I = set()
for elem in incidents:
    I.update(elem['units'])

I = list(I)

In [93]:
N = list(neighborhood.nta)
J = list(distance.columns)
K= [1,2]

In [94]:
X = {}
for i in I:
    for j in J:
        for k in K:
            v = solver.NumVar(0, solver.infinity(), "Number of vehicles of type " + str(i) + " deployed at station "+ str(j)+ " during shift " + str(k))
            X[(i,j,k)] = v

In [98]:
Y = {}
for i in I:
    for a in J:
        for b in J:
            v = solver.NumVar(0, solver.infinity(), "Number of vehicles of type " + str(i) + " moved from station "+ str(a)+ " to station " + str(b))
            Y[(i,a,b)] = v

In [107]:
T = {}
for i in I:
    for j in J:
        for k in K:
            v = solver.BoolVar("if a vehicle of type" + str(i)  + " is deployed in station " + str(j) + " during shift " + str(k))
            T[(i,j,k)] = v

In [111]:
Q = {}
for j in J:
    for k in K:
        v = solver.BoolVar("if there are two or more vehicles in station " + str(j) + " during shift " + str(k))
        Q[(j,k)] = v

## Creting constraints

### 1. Station capacity

\begin{equation}
    \sum_{i \in I}^{}X_{i, j, k} \leq c_{j}, \forall j \in J, k \in K
    \label{station_capacity}
\end{equation}

In [117]:
response2 = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/located_firehouses.json")
stations = json.loads(response2.text)

In [133]:
capacities = [(feature['properties']['capacity'], feature['properties']['FacilityAddress']) for feature in stations['features']]
capacities

[(2, '42 South Street'),
 (1, '49 Beekman Street'),
 (4, '100 Duane Street'),
 (1, '14 N. Moore Street'),
 (2, '75 Canal Street'),
 (3, '25 Pitt Street'),
 (2, '222 East 2 Street'),
 (1, '340 East 14 Street'),
 (2, '253 Lafayette Street'),
 (2, '42 Great Jones Street'),
 (2, '108 East 13 Street'),
 (1, '132 West 10 Street'),
 (2, '440 West 38th Street'),
 (1, '220 West 37 Street'),
 (3, '150 West 19 Street'),
 (2, '142 West 31 Street'),
 (1, '14 East 18 Street'),
 (2, '234 East 29 Street'),
 (1, '238 East 40 Street'),
 (3, '782 Eighth Avenue'),
 (1, '215 W 58th Street'),
 (2, '131 Amsterdam Avenue'),
 (2, '207 West 77th Street'),
 (1, '120 West 83 Street'),
 (3, '165 East 51 Street'),
 (1, '221 East 75 Street'),
 (3, '159 East 85 Street'),
 (2, '1367 Fifth Avenue'),
 (2, '1836 Third Avenue'),
 (1, '242 East 111 Street'),
 (3, '2282 Third Avenue'),
 (3, '145 West 100th St.'),
 (1, '500 West 113 Street'),
 (2, '111 West 133 Street'),
 (2, '415 West 125 Street'),
 (2, '513 West 161 Street

In [121]:
capacity_constraints = []
for feature in stations['features']:
    capacity = feature['properties']['capacity']
    c = solver.Constraint(-solver.infinity(), capacity)
    capacity_constraints.append(c)

In [123]:
capacity_constraints

[<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000002BD82D0EA20> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000002BD82D0E570> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000002BD82DFD2A0> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000002BD82DFD300> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000002BD82DFD570> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000002BD82DFD960> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000002BD82DFD6C0> >,
 <ortools.linear_solver.pyw

$mixing: 2 A + 4 B \leq 11$

$pigmentation: 3A + 1 B \leq 9 $

$mixing: -\infty \leq 2 A + 4 B \leq 11$

$pigmentation: -\infty \leq 3A + 1 B \leq 9 $

### Objective Function


$max Z: \sum_{k=1}^{2} \sum_{i=1}^{5} \sum_{j=1}^{218} Y_{i,j,k} \left(
\sum_{i=1}^{195} \cdot s_{j,n,k,i} \cdot (0.4p_n + 0.1h_{n,i,k} - 0.5d_{n,j} \cdot j) \right)$